In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
from keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the path to your data directory
data_directory = '/content/drive/My Drive/data'
# Function to calculate SDNN from heart rate data
def calculate_sdnn(heart_rate_values, window_size=5):
    # Convert heart rate to RR intervals (assuming constant heart rate for simplicity)
    rr_intervals = 60 / heart_rate_values  # since values are in bpm

    # Calculate SDNN over a moving window
    sdnn_values = []
    for i in range(len(rr_intervals) - window_size + 1):
        sdnn_values.append(np.std(rr_intervals[i:i+window_size]))

    return np.array(sdnn_values)

# Modified function to load and preprocess heart rate data
def load_and_preprocess_heartrate(subject_id):
    file_path = os.path.join(data_directory, 'heart_rate', f'{subject_id}_heartrate.txt')
    data = pd.read_csv(file_path, header=None, names=['date', 'heartrate'])
    heartrate_values = data['heartrate']

    # Calculate SDNN from heart rate data
    sdnn_values = calculate_sdnn(heartrate_values)

    return sdnn_values

def load_labeled_sleep(subject_id):
    file_path = os.path.join(data_directory, 'labels', f'{subject_id}_labeled_sleep.txt')
    data = pd.read_csv(file_path, header=None, delim_whitespace=True, names=['date', 'stage'])
    # Encode sleep labels as binary values: 1 for awake (label=0), 0 for asleep (labels 1-5)
    return (data['stage'] == 0).astype(int).values

subject_ids = [file.split('_')[0] for file in os.listdir(os.path.join(data_directory, 'labels')) if '_labeled_sleep.txt' in file]

X_heartrate = []
y_sleep = []

sequence_length = 200

for subject_id in subject_ids:
    heartrate_values = load_and_preprocess_heartrate(subject_id)
    sleep_labels = load_labeled_sleep(subject_id)

    for i in range(0, len(heartrate_values) - sequence_length):
        X_heartrate.append(heartrate_values[i:i+sequence_length])
        y_sleep.append(sleep_labels[min(i+sequence_length-1, len(sleep_labels)-1)])

In [4]:
X_heartrate = np.array(X_heartrate)
y_sleep = np.array(y_sleep)

label_encoder = LabelEncoder()
y_sleep = label_encoder.fit_transform(y_sleep)

scaler = StandardScaler()
X_heartrate_scaled = scaler.fit_transform(X_heartrate)

X_train, X_test, y_train, y_test = train_test_split(
    X_heartrate_scaled, y_sleep, test_size=0.1, random_state=42
)

# Model architecture
num_classes = 1  # Binary classification: awake or asleep

In [5]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

model = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),  # Convolutional layer
    MaxPooling1D(pool_size=2),  # Max pooling layer
    Conv1D(64, kernel_size=3, activation='relu'),  # Convolutional layer
    MaxPooling1D(pool_size=2),  # Max pooling layer
    Flatten(),  # Flatten layer
    Dense(64, activation='relu'),  # Dense layer
    Dense(1, activation='sigmoid')  # Output layer with sigmoid activation for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
from keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train[..., np.newaxis],  # Expand dimension for Conv1D input shape
    y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

# Model evaluation
y_pred = model.predict(X_test[..., np.newaxis])
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes
accuracy = accuracy_score(y_test, y_pred_classes)
print("Test Accuracy:", accuracy)

Epoch 1/20
6281/6281 [==============================] - 90s 14ms/step - loss: 0.5034 - accuracy: 0.7777 - val_loss: 0.4402 - val_accuracy: 0.8111
Epoch 2/20
6281/6281 [==============================] - 89s 14ms/step - loss: 0.3703 - accuracy: 0.8415 - val_loss: 0.3124 - val_accuracy: 0.8703
Epoch 3/20
6281/6281 [==============================] - 88s 14ms/step - loss: 0.2632 - accuracy: 0.8910 - val_loss: 0.2385 - val_accuracy: 0.9027
Epoch 4/20
6281/6281 [==============================] - 86s 14ms/step - loss: 0.2024 - accuracy: 0.9171 - val_loss: 0.1966 - val_accuracy: 0.9195
Epoch 5/20
6281/6281 [==============================] - 86s 14ms/step - loss: 0.1644 - accuracy: 0.9331 - val_loss: 0.1802 - val_accuracy: 0.9270
Epoch 6/20
6281/6281 [==============================] - 85s 14ms/step - loss: 0.1385 - accuracy: 0.9438 - val_loss: 0.1641 - val_accuracy: 0.9343
Epoch 7/20
6281/6281 [==============================] - 87s 14ms/step - loss: 0.1200 - accuracy: 0.9523 - val_loss: 0.1728 -